<a href="https://colab.research.google.com/github/johnnyff/bigcontest2021/blob/main/danm_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import warnings
from tqdm import tqdm
import torch
import datetime as dt
import torch.nn as nn

warnings.filterwarnings(action='ignore') 

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/bigcontest/data/bigcontest_dataset.xlsx', header = 1)
df

In [ ]:
columns = ['num','year','month','day','hour','target',
           'average_rain_1','a_rain_1','b_rain_1','c_rain_1','d_rain_1','e_level_1','d_level_1',
           'average_rain_2','a_rain_2','b_rain_2','c_rain_2','d_rain_2','e_level_2','d_level_2',
           'average_rain_3','a_rain_3','b_rain_3','c_rain_3','d_rain_3','e_level_3','d_level_3',
           'average_rain_4','a_rain_4','b_rain_4','c_rain_4','d_rain_4','e_level_4','d_level_4',
           'average_rain_5','a_rain_5','b_rain_5','c_rain_5','d_rain_5','e_level_5','d_level_5',
           'average_rain_6','a_rain_6','b_rain_6','c_rain_6','d_rain_6','e_level_6','d_level_6'
           ]

In [ ]:
df.columns =columns


In [ ]:
df.drop(['year','month','day','hour'],axis =1 , inplace = True)
df

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device =='cuda':
    torch.cuda.manual_seed_all(777)
    print(torch.cuda.get_device_name(0))

In [ ]:
import torch
from torch.nn import Module, LSTM, Linear
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from torch.autograd import Variable

In [ ]:
train_data = df[:-160]
train_data[train_data['num']==1]

### Kalman filter

In [ ]:
!pip install filterpy

In [ ]:
import numpy as np
from filterpy.kalman import KalmanFilter
from filterpy.common import Q_discrete_white_noise

my_filter = KalmanFilter(dim_x=2,dim_z=1) #create kalman filter

my_filter.x = np.array([[2.],
                [0.]])       # initial state (location and velocity)

my_filter.F = np.array([[1.,1.],
                [0.,1.]])    # state transition matrix

my_filter.H = np.array([[1.,0.]])    # Measurement function
my_filter.P *= 1000.                 # covariance matrix
my_filter.R = 5                      # state uncertainty
my_filter.Q = Q_discrete_white_noise(dim = 2,dt=.1,var=.1) # process uncertainty

In [ ]:
for i in range(2,44):
  z = train_data.iloc[:,i] #필터링 할 피쳐   
  a = []           #필터링 된 피쳐(after)
  b = []           #필터링 전 피쳐(before)
  my_filter = KalmanFilter(dim_x=2,dim_z=1) #create kalman filter
  my_filter.x = np.array([[2.],[0.]])       # initial state (location and velocity)
  my_filter.F = np.array([[1.,1.], [0.,1.]])    # state transition matrix
  my_filter.H = np.array([[1.,0.]])    # Measurement function
  my_filter.P *= 1000.                 # covariance matrix
  my_filter.R = 5                      # state uncertainty
  my_filter.Q = Q_discrete_white_noise(dim = 2,dt=.1,var=.1) # process uncertainty   
  for z in z:
      my_filter.predict()
      my_filter.update(z)
      # do something with the output
      x = my_filter.x
      a.extend(x[0])
      b.append(z)
  train_data.iloc[:,i]=a
train_data

In [ ]:
scaler = MinMaxScaler()
scaler.fit(train_data.iloc[:,1:])
scaler

In [ ]:
def sliding_windows(data, seq_length):
    x = []
    y = []
    for j in range(1,26):
      part = data[data['num']==j]
      part.drop(['num'], axis =1 , inplace =True)
      part = scaler.transform(part)
      temp =[]
      for i in range(seq_length):
        temp.append(part[0,1:])

      for i in range(seq_length -1):
        x.append(temp)
        y.append(part[0,0])

      for i in range(len(part)-seq_length):
      
        _x = part[i:(i+seq_length),1:]
        _y = part[i+seq_length-1,0]
        x.append(_x)
        y.append(_y)
    return np.array(x),np.array(y)

In [ ]:
seq_length = 3
x, y = sliding_windows(train_data, seq_length)




In [ ]:
x.shape

In [ ]:
y.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1422)
print(X_train.shape, X_test.shape)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
X_train_t = torch.Tensor(X_train)
X_test_t = torch.Tensor(X_test)
y_train_t = torch.Tensor(y_train)
y_test_t = torch.Tensor(y_test)

In [ ]:
train_batch_size = 5
test_batch_size =1 
train_loader = DataLoader(TensorDataset(X_train_t, y_train_t), shuffle=True, batch_size=train_batch_size, num_workers = 3)
test_loader = DataLoader(TensorDataset(X_test_t, y_test_t), shuffle=False, batch_size=test_batch_size,num_workers = 3)

Modeling

In [ ]:

class LSTM(nn.Module):
    def __init__(self, input_size=42, hidden_layer_size=120, layer_dim =1, output_size=1,dropout=0.25):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size
        self.layer_dim = layer_dim

        self.lstm = nn.LSTM(input_size, hidden_layer_size, layer_dim, batch_first = True,bidirectional = True)

        self.fc1 = nn.Linear(2*hidden_layer_size, hidden_layer_size)
        # self.batch1 = nn.BatchNorm1d(hidden_layer_size)
        self.relu = nn.ReLU()
        
        self.fc2 = nn.Linear(hidden_layer_size,1)
    
        self.dropout = nn.Dropout(dropout)
        # self.hidden_cell = (torch.zeros(2*1,5,self.hidden_layer_size),
        #                     torch.zeros(2*1,5,self.hidden_layer_size))
        
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)


    def forward(self, input_seq):

        h0 = torch.zeros(self.layer_dim, input_seq.size(0), self.hidden_layer_size).to(device)
        # Initialize cell state
        c0 = torch.zeros(self.layer_dim, input_seq.size(0), self.hidden_layer_size).to(device)

        batch_size = input_seq.size(0)
   
        lstm_out, self.hidden_cell = self.lstm(input_seq)
        # print(lstm_out.shape)
        out = self.fc1(lstm_out)
        # print(out.shape)
        # out = out.permute(0, 2, 1)
        # out = self.batch1(out)
        # print(out.shape)
        # out = out.permute(0,2,1)
        out = self.relu(out)
        # print(out.shape)
        out = self.fc2(out)
        # print(out.shape)
        out = out.view(batch_size,-1)
        # print(out.shape)
        out = out[:,-1]
        # print(out.shape)

        return out


In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size=42, hidden_layer_size=120, layer_dim =1, output_size=1,dropout=0.25):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size
        self.layer_dim = layer_dim

        self.lstm = nn.LSTM(input_size, hidden_layer_size, layer_dim, batch_first = True,bidirectional = True)

        self.fc1 = nn.Linear(2*hidden_layer_size, hidden_layer_size)
        self.fc2 = nn.Linear(hidden_layer_size,1)
        self.dropout = nn.Dropout(dropout)
        # self.hidden_cell = (torch.zeros(2*1,5,self.hidden_layer_size),
        #                     torch.zeros(2*1,5,self.hidden_layer_size))
        
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)


    def forward(self, input_seq):

        # h0 = torch.zeros(self.layer_dim, input_seq.size(0), self.hidden_layer_size).requires_grad_().to(device)
        # # Initialize cell state
        # c0 = torch.zeros(self.layer_dim, input_seq.size(0), self.hidden_layer_size).requires_grad_().to(device)

        batch_size = input_seq.size(0)
   
        lstm_out, self.hidden_cell = self.lstm(input_seq)
        # print(lstm_out.view(len(input_seq),-1)[:,-1].shape)
        # print(lstm_out.shape)
        out = self.fc1(lstm_out)
        out = self.fc2(out)
        out = out.view(batch_size,-1)
        # print(out.shape)
        out = out[:,-1]
        # print(out.shape)

        return out

In [ ]:
model = LSTM().to(device)
loss_function = nn.MSELoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
model

In [ ]:
# test
for seq, labels in train_loader:
  seq = seq.to(device)
  labels = labels.to(device)
  res = model(seq)
  print(res.shape)
  break


In [ ]:
epoch_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 250, gamma=0.8, verbose = True)


### Train

In [ ]:
epochs = 2000
patience = 100
min_val_loss = 9999
counter = 0
batch_loss = []
val_loss = []
preds =[]
gts = []
for i in range(epochs):
    mse_train =0 
    for seq, labels in train_loader:
        seq = seq.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        model.hidden_cell = (torch.zeros(2, seq.size(0), model.hidden_layer_size),
                        torch.zeros(2, seq.size(0), model.hidden_layer_size))

        y_pred = model(seq)
        loss = loss_function(y_pred, labels)
        loss.backward()
        mse_train+= loss.item()
        optimizer.step()
    batch_loss.append(mse_train)
    epoch_scheduler.step()

   
    if (i%5==0):
      with torch.no_grad():
          mse_val = 0
          total = 1
          for seq, labels in test_loader:
              total = len(test_loader)
              seq = seq.to(device)
              labels = labels.to(device).view([-1,seq.size(0)])
              y_pred = model(seq).view([-1,seq.size(0)])

              labels = torch.tensor(labels)
              y_pred = torch.tensor(y_pred)
              
              temp = (seq[:,len(seq)-1,:])
              gt = scaler.inverse_transform(torch.cat([labels,temp], dim =1).detach().cpu().clone().numpy())[0][0]
              pred = scaler.inverse_transform(torch.cat([y_pred,temp], dim =1).detach().cpu().clone().numpy())[0][0]
              loss = loss_function(torch.tensor(pred), torch.tensor(gt)).item()
              gts.append(gt)
              preds.append(pred)              
              mse_val += loss

          val_loss.append((mse_val/total)**0.5)

      print("iteration : {} , rsme : {}".format(i, (mse_val/total)**0.5))
      if min_val_loss > mse_val**0.5:
          min_val_loss = mse_val**0.5
          print("Saving...")
          torch.save(model.state_dict(), "/content/drive/MyDrive/bigcontest/danm_lstm_model_kalma.pt")
          counter = 0
      else: 
          counter += 1
      
      if counter == patience:
          break
    
    

In [ ]:
print(min(val_loss))
plt.plot(val_loss)

In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/bigcontest/danm_lstm_model_kalma.pt'))
model


In [ ]:
model.eval()
with torch.no_grad():
    gts = []
    preds = []
    mse_val = 0
    total = 0
    for seq, labels in test_loader:
        total = len(test_loader)
        seq = seq.to(device)
        labels = labels.to(device).view([-1,seq.size(0)])
        y_pred = model(seq).view([-1,seq.size(0)])

        labels = torch.tensor(labels)
        y_pred = torch.tensor(y_pred)

        temp = (seq[:,len(seq)-1,:])
        gt = scaler.inverse_transform(torch.cat([labels,temp], dim =1).detach().cpu().clone().numpy())[0][0]
        pred = scaler.inverse_transform(torch.cat([y_pred,temp], dim =1).detach().cpu().clone().numpy())[0][0]
        loss = loss_function(torch.tensor(pred), torch.tensor(gt)).item()
        gts.append(gt)
        preds.append(pred)              
        mse_val += loss
    

In [ ]:
feed = pd.DataFrame({'predict': preds, 'GT ' : gts})
feed

In [ ]:
max(abs(feed['GT ']-feed['predict']))

In [ ]:
(mse_val/574)**0.5

In [ ]:
plt.plot(gts, label ='gt')
plt.plot(preds,label = 'pred')
plt.legend()
plt.show()

In [ ]:
sns.scatterplot(preds, gts)

In [ ]:
feed[(abs(feed['GT ']-feed['predict']))>300]

In [ ]:
train_data[(train_data['target']>200.34) & (train_data['target']<200.35)]

In [ ]:
train_data[(train_data['target']>368) & (train_data['target']<369)]

In [ ]:
train_data[train_data['num']==3]

In [ ]:
train_data[train_data['num']==25]

### Check with each section

In [ ]:
fig, ax = plt.subplots(5,5, figsize=(28,56))
ax = ax.flatten()

start_point = 0
end_point = 0 
for i in range(1,26):
  length = len(train_data[train_data['num']==i])
  end_point = start_point + length
  x_test1 = torch.Tensor(x[start_point:end_point])
  y_test1 = torch.Tensor(y[start_point:end_point])
  start_point = end_point
  test1_loader = DataLoader(TensorDataset(x_test1, y_test1), shuffle=False, batch_size=test_batch_size,num_workers = 3)

  model.eval()
  with torch.no_grad():
      gts_c = []
      preds_c = []
      mse_val_c = 0
      total = 0
      for seq, labels in test1_loader:
          total = len(test_loader)
          seq = seq.to(device)
          labels = labels.to(device).view([-1,seq.size(0)])
          y_pred = model(seq).view([-1,seq.size(0)])

          labels = torch.tensor(labels)
          y_pred = torch.tensor(y_pred)

          temp = (seq[:,len(seq)-1,:])
          gt = scaler.inverse_transform(torch.cat([labels,temp], dim =1).detach().cpu().clone().numpy())[0][0]
          pred = scaler.inverse_transform(torch.cat([y_pred,temp], dim =1).detach().cpu().clone().numpy())[0][0]
          loss = loss_function(torch.tensor(pred), torch.tensor(gt)).item()

          gts_c.append(gt)
          preds_c.append(pred)              
          mse_val_c += loss

          ax[i-1].plot(gts_c, label ='gt')
          ax[i-1].plot(preds_c,label = 'pred')
          ax[i-1].set(title = "{} sector".format(i))
plt.tight_layout()
    

In [ ]:
feed_c= pd.DataFrame({'predict': preds_c, 'GT ' : gts_c})
feed_c

In [ ]:
plt.plot(gts_c, label ='gt')
plt.plot(preds_c,label = 'pred')
plt.legend()
plt.show()